# Open Metro and AeroDataBox API Integration

The purpose of this section is to explore the integration of Open Metro and AeroDataBox APIs for retrieving weather and flight data. The methodology that I used to explore these APIs is as follows:
* Create aeromarket-api and openmeteo-api packages to interact with the respective APIs more easily.
* Use environment variables to securely store API keys and URLs.
* Implement classes and methods to fetch and process data from both APIs. 

In [3]:
# Executing this cell does some magic
%load_ext autoreload
%autoreload 2

In [14]:
from openmeteoapi.WeatherData import Weather
from openmeteoapi.APICaller import OpenMeteoAPICaller
from aeroapi_market.APICaller import APICaller
from aeroapi_market.Flights import Flights
import pandas as pd
import os
from dotenv import load_dotenv

As a reminder, our goal is to get a dataset with the following format:

In [ ]:
df_full = pd.read_parquet("https://drive.google.com/file/d/1vitmLije5W6NSJ-jswv_nz2kgLPUcrDO/view?usp=sharing")
df_full.head()

c:\Users\rashd\AppData\Local\Programs\Python\Python313\Lib\site-packages\gdown\parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1vitmLije5W6NSJ-jswv_nz2kgLPUcrDO
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1vitmLije5W6NSJ-jswv_nz2kgLPUcrDO/view?usp=sharing
To: d:\OneDrive\Personal_Things\My Website\My website HTML\backend\supplychainAPI\notebooks\file.parquet
110kB [00:00, 16.5MB/s]


ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

# Weather Data Retrieval using OpenMeteo API

To retrieve weather data, I created a `Weather` class in the `openmeteo-api` package. This class uses an `APICaller` to make requests to the OpenMeteo API and fetch historical weather data based on specified parameters such as latitude, longitude, start date, end date, and hourly variables. These data will be very helpful for the model to predict flight delays based on weather conditions.


I defined the disiered weather variables in the weather class for easier readability. Alternatively, you can specify the variables directly when calling the methods. The desired weather variables are as follows:
- Temperature at 2 meters (°C)
- Precipitation (mm)
- Wind Speed at 10 meters (km/h)
- Humidity at 2 meters (%)
- rainfall (mm)
- wind gusts at 10 meters (km/h)
- cloud cover (%)
- cloud cover low (%)
- apparent temperature (°C)
- surface pressure (hPa)
- pressure at mean sea level (hPa)

In order to retrieve weather data, we need to create an instance of the 'Weather' class and call 'to_hourly_dataframe' method (or to_daily_dataframe) to get the hourly desired data in a pandas DataFrame format. As follows:

In [20]:
import airportsdata

airports = airportsdata.load("ICAO")  # load ICAO-indexed data

icao = "KJFK"
airport = airports.get(icao)

if airport:
    lat = airport["lat"]
    lon = airport["lon"]
    print(lat, lon)


40.63993 -73.77869


In [3]:
import requests

url = "https://archive-api.open-meteo.com/v1/archive"

lats = [40.6413, 34.0522, 51.4700]
lons = [-73.7781, -118.2437, -0.4543]

params = {
    "latitude": ",".join(map(str, lats)),
    "longitude": ",".join(map(str, lons)),
    "start_date": "2023-01-01",
    "end_date": "2023-01-31",
    "hourly": "temperature_2m,relativehumidity_2m",
    "timezone": "auto",
}

response = requests.get(url, params=params)
data = response.json()

In [4]:
data

[{'latitude': 40.667835,
  'longitude': -73.81021,
  'generationtime_ms': 0.5044937133789062,
  'utc_offset_seconds': -18000,
  'timezone': 'America/New_York',
  'timezone_abbreviation': 'GMT-5',
  'elevation': 3.0,
  'hourly_units': {'time': 'iso8601',
   'temperature_2m': '°C',
   'relativehumidity_2m': '%'},
  'hourly': {'time': ['2023-01-01T00:00',
    '2023-01-01T01:00',
    '2023-01-01T02:00',
    '2023-01-01T03:00',
    '2023-01-01T04:00',
    '2023-01-01T05:00',
    '2023-01-01T06:00',
    '2023-01-01T07:00',
    '2023-01-01T08:00',
    '2023-01-01T09:00',
    '2023-01-01T10:00',
    '2023-01-01T11:00',
    '2023-01-01T12:00',
    '2023-01-01T13:00',
    '2023-01-01T14:00',
    '2023-01-01T15:00',
    '2023-01-01T16:00',
    '2023-01-01T17:00',
    '2023-01-01T18:00',
    '2023-01-01T19:00',
    '2023-01-01T20:00',
    '2023-01-01T21:00',
    '2023-01-01T22:00',
    '2023-01-01T23:00',
    '2023-01-02T00:00',
    '2023-01-02T01:00',
    '2023-01-02T02:00',
    '2023-01-02T03:00

In [34]:
api_caller_weather = OpenMeteoAPICaller()
weather_data = Weather(
    api_caller=api_caller_weather,
    airport_code="KJFK",
    start_date="2024-12-11",
    end_date="2024-12-25",
    code_type="ICAO",
)
weather_df = weather_data.to_hourly_dataframe()
weather_df

,date,snowfall,rain,precipitation,wind_speed_10m,wind_gusts_10m,cloud_cover_low,cloud_cover,temperature_2m,apparent_temperature,surface_pressure,relative_humidity_2m,pressure_msl,queried_airport_code
0,2024-12-11 00:00:00-08:00,0.0,0.0,0.0,0.969330,3.600000,0.0,100.0,2.85,0.538085,993.670837,79.766556,1026.699951,KJFK
1,2024-12-11 01:00:00-08:00,0.0,0.0,0.0,1.548419,3.240000,0.0,100.0,2.20,-0.233988,993.110535,82.029907,1026.199951,KJFK
2,2024-12-11 02:00:00-08:00,0.0,0.0,0.0,1.800000,4.320000,0.0,100.0,2.05,-0.457596,992.899109,81.413658,1026.000000,KJFK
3,2024-12-11 03:00:00-08:00,0.0,0.0,0.0,2.036468,4.680000,0.0,100.0,2.60,0.014548,992.673584,76.591270,1025.699951,KJFK
4,2024-12-11 04:00:00-08:00,0.0,0.0,0.0,1.207477,4.680000,0.0,100.0,2.95,0.449912,992.327515,73.358429,1025.300049,KJFK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2024-12-25 19:00:00-08:00,0.0,0.0,0.0,5.920878,9.360000,0.0,97.0,8.00,5.665333,990.584351,74.538841,1022.900024,KJFK
356,2024-12-25 20:00:00-08:00,0.0,0.0,0.0,2.885619,11.879999,0.0,100.0,7.05,5.157882,991.445190,79.530273,1023.900024,KJFK
357,2024-12-25 21:00:00-08:00,0.0,0.0,0.0,5.447788,9.720000,0.0,100.0,6.45,4.203760,991.086243,83.459351,1023.599976,KJFK
358,2024-12-25 22:00:00-08:00,0.0,0.0,0.0,4.155622,9.720000,0.0,100.0,6.15,4.121793,991.536377,86.104759,1024.099976,KJFK


In [66]:
airports = ["JFK", "LAX", "LHR", "CDG", "HND", "DXB", "SIN", "SYD", "FRA", "AMS"]
api_caller_weather = OpenMeteoAPICaller()
weather = Weather(
    api_caller=api_caller_weather,
    airport_code=airports,
    start_date="2024-12-11",
    end_date="2024-12-25",
    code_type="IATA",
)
hourly = weather.fetch()
print(hourly[0].Timezone())

b'America/New_York'


Similarly, we can request weather data for multiple locations:

In [39]:
airports = ["JFK", "LAX", "LHR", "CDG", "HND", "DXB", "SIN", "SYD", "FRA", "AMS"]
airport_list = list(airports)
weather = Weather(
    api_caller=api_caller_weather,
    airport_code=airport_list,
    start_date="2024-12-11",
    end_date="2024-12-25",
    code_type="IATA",
)
weather_df2 = weather.to_hourly_dataframe()
weather_df2

,date,snowfall,rain,precipitation,wind_speed_10m,wind_gusts_10m,cloud_cover_low,cloud_cover,temperature_2m,apparent_temperature,surface_pressure,relative_humidity_2m,pressure_msl,queried_airport_code
0,2024-12-11 00:00:00-05:00,0.0,0.1,0.1,16.201000,33.480000,100.0,100.0,11.75,9.913984,1013.135620,99.670563,1013.500000,JFK
1,2024-12-11 01:00:00-05:00,0.0,0.7,0.7,15.865546,33.839996,100.0,100.0,11.55,9.700219,1012.035706,99.670036,1012.400024,JFK
2,2024-12-11 02:00:00-05:00,0.0,0.7,0.7,19.937794,39.959999,73.0,100.0,12.00,9.650640,1011.436462,98.690514,1011.799988,JFK
3,2024-12-11 03:00:00-05:00,0.0,1.2,1.2,19.408306,39.239998,0.0,100.0,12.85,10.625551,1011.337463,94.262718,1011.700012,JFK
4,2024-12-11 04:00:00-05:00,0.0,0.3,0.3,14.458382,38.880001,0.0,100.0,13.20,11.493306,1010.438416,88.260132,1010.799988,JFK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2024-12-25 19:00:00+01:00,0.0,0.0,0.0,8.131936,16.559999,100.0,100.0,8.85,7.270769,1034.048218,97.661545,1034.800049,AMS
3596,2024-12-25 20:00:00+01:00,0.0,0.0,0.0,7.100310,15.119999,100.0,100.0,8.75,7.295176,1034.447510,97.659706,1035.199951,AMS
3597,2024-12-25 21:00:00+01:00,0.0,0.0,0.0,6.766180,13.320000,100.0,100.0,8.65,7.230936,1034.647095,97.989471,1035.400024,AMS
3598,2024-12-25 22:00:00+01:00,0.0,0.0,0.0,7.329338,13.679999,100.0,100.0,8.70,7.211770,1034.647461,97.990250,1035.400024,AMS


# AeroDataBox Flight Data Retrieval

To retrive flight data, I created a `Flights` class in the `aeromarket-api` package. This class uses an `APICaller` to make requests to the AeroDataBox API and fetch flight data based on specified parameters such as flight number, date, and airport code. These data will be very helpful for the model to predict flight delays based on various factors.

The first step is to set up the API key and base URL for the AeroDataBox API. This can be done using environment variables for security purposes. Here is an example of how to set up the API key and base URL:

In [5]:
load_dotenv()
API_KEY = os.getenv("AERODATABOX_API_KEY")
BASE_URL = os.getenv("AERODATABOX_BASE_URL")

In [6]:
api_caller = APICaller(BASE_URL, API_KEY)
flights = Flights(api_caller, 
    airport_code="ATL",
    from_local="2025-10-26T00:00",
    to_local="2025-10-26T12:00",
    keep_cols=True
    )

We can see below that flight.get_airport_flights method retrieves all departed and arrived flights for a specific airport on a given date. It returns the data in a json format that can be further processed or analyzed.

In [8]:
flights.get_airport_flights(code_type="iata")

{'departures': [{'movement': {'airport': {'icao': 'RKSI',
     'iata': 'ICN',
     'name': 'Seoul',
     'timeZone': 'Asia/Seoul'},
    'scheduledTime': {'utc': '2025-10-26 03:50Z',
     'local': '2025-10-25 23:50-04:00'},
    'revisedTime': {'utc': '2025-10-26 04:07Z',
     'local': '2025-10-26 00:07-04:00'},
    'runwayTime': {'utc': '2025-10-26 04:07Z',
     'local': '2025-10-26 00:07-04:00'},
    'terminal': 'I',
    'quality': ['Basic', 'Live']},
   'number': 'DL 27',
   'callSign': 'DAL27',
   'status': 'Departed',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'reg': 'N527DN',
    'modeS': 'A6A347',
    'model': 'Airbus A350-900'},
   'airline': {'name': 'Delta Air Lines', 'iata': 'DL', 'icao': 'DAL'}},
  {'movement': {'airport': {'icao': 'KBOS',
     'iata': 'BOS',
     'name': 'Boston',
     'timeZone': 'America/New_York'},
    'scheduledTime': {'utc': '2025-10-26 02:35Z',
     'local': '2025-10-25 22:35-04:00'},
    'revisedTime': {'utc': '2025-10-26

flights.getairport_flights_df format the retrived data nicely into a pandas DataFrame for easier analysis and manipulation.

In [12]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns in the DataFrame
flights_df = flights.get_airport_flights_df()
pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
display(flights_df)

,direction,flight_number,callsign,status,codeshare_status,airline,airline_iata,aircraft_model,aircraft_reg,airport_icao,airport_iata,airport_name,timezone,scheduled_utc,scheduled_local,actual_utc,actual_local,terminal,runway,is_cargo,quality,date,queried_airport_iata
0,Arrival,F9 1241,FFT1241,Arrived,IsOperator,Frontier,F9,Airbus A321,N717FR,KMCO,MCO,Orlando,America/New_York,2025-10-26 04:18:00+00:00,2025-10-26 00:18:00-04:00,2025-10-26 04:01:00+00:00,2025-10-26 00:01:00-04:00,N,09R,False,"Basic, Live",2025-10-26 00:00:00-04:00,ATL
1,Arrival,DL 748,DAL9964,Arrived,IsOperator,Delta Air Lines,DL,Airbus A321 NEO,N534DT,KLAS,LAS,Las Vegas,America/Los_Angeles,2025-10-25 23:27:00+00:00,2025-10-25 19:27:00-04:00,2025-10-26 04:01:00+00:00,2025-10-26 00:01:00-04:00,S,None,False,"Basic, Live",2025-10-25 19:00:00-04:00,ATL
2,Arrival,DL 436,DAL436,Arrived,IsOperator,Delta Air Lines,DL,Airbus A320,N347NW,KDFW,DFW,Dallas-Fort Worth,America/Chicago,2025-10-26 03:12:00+00:00,2025-10-25 23:12:00-04:00,2025-10-26 04:02:00+00:00,2025-10-26 00:02:00-04:00,S,08L,False,"Basic, Live",2025-10-25 23:00:00-04:00,ATL
3,Arrival,YV 4049,ASH4049,Arrived,IsOperator,Mesa Airlines,YV,Embraer 175,N88327,KIAH,IAH,Houston,America/Chicago,2025-10-26 04:01:00+00:00,2025-10-26 00:01:00-04:00,2025-10-26 04:13:00+00:00,2025-10-26 00:13:00-04:00,None,09R,False,"Basic, Live",2025-10-26 00:00:00-04:00,ATL
4,Arrival,QR 8170,QTR8170,Arrived,IsOperator,Qatar Airways,QR,Boeing 777,A7-BFX,EDDF,FRA,Frankfurt-am-Main,Europe/Berlin,2025-10-26 04:49:00+00:00,2025-10-26 00:49:00-04:00,2025-10-26 04:53:00+00:00,2025-10-26 00:53:00-04:00,None,None,False,"Basic, Live",2025-10-26 00:00:00-04:00,ATL
5,Arrival,NK 130,NKS130,Arrived,IsOperator,Spirit,NK,Airbus A321 NEO,N716NK,KFLL,FLL,Fort Lauderdale,America/New_York,2025-10-26 04:39:00+00:00,2025-10-26 00:39:00-04:00,2025-10-26 05:05:00+00:00,2025-10-26 01:05:00-04:00,N,None,False,"Basic, Live",2025-10-26 00:00:00-04:00,ATL
6,Arrival,8C 3305,ATN3305,Expected,IsOperator,ATI,8C,Boeing 767-300,N351CM,KILN,ILN,Wilmington,America/New_York,2025-10-26 05:31:00+00:00,2025-10-26 01:31:00-04:00,2025-10-26 05:31:00+00:00,2025-10-26 01:31:00-04:00,None,None,False,"Basic, Live",2025-10-26 01:00:00-04:00,ATL
7,Arrival,DL 8902,DAL8902,Arrived,IsOperator,Delta Air Lines,DL,Boeing 757-300,N591NW,KCMI,CMI,Champaign/Urbana,America/Chicago,2025-10-26 06:49:00+00:00,2025-10-26 02:49:00-04:00,2025-10-26 06:56:00+00:00,2025-10-26 02:56:00-04:00,None,None,False,"Basic, Live",2025-10-26 02:00:00-04:00,ATL
8,Arrival,9E 8902,DAL8902,Arrived,IsCodeshared,Endeavor Air,9E,Boeing 757-300,N591NW,KCMI,CMI,Champaign/Urbana,America/Chicago,2025-10-26 06:49:00+00:00,2025-10-26 02:49:00-04:00,2025-10-26 06:56:00+00:00,2025-10-26 02:56:00-04:00,None,None,False,"Basic, Live",2025-10-26 02:00:00-04:00,ATL
9,Arrival,F9 3216,FFT3216,Arrived,IsOperator,Frontier,F9,Airbus A321 NEO,N630FR,KLAX,LAX,Los Angeles,America/Los_Angeles,2025-10-26 09:01:00+00:00,2025-10-26 05:01:00-04:00,2025-10-26 08:48:00+00:00,2025-10-26 04:48:00-04:00,N,08L,False,"Basic, Live",2025-10-26 05:00:00-04:00,ATL


Now we can easily join the weather data and flight data based on the airport code and date to create a comprehensive dataset for analysis.

In [82]:
start_date = min(flights_df['date']).strftime('%Y-%m-%d')
end_date = max(flights_df['date']).strftime('%Y-%m-%d')
airport_code = flights_df['queried_airport_iata'].unique()[0]
weather_data = Weather(
    api_caller=api_caller_weather,
    airport_code=airport_code,
    code_type="iata",
    start_date=start_date,
    end_date=end_date,
)
merged_df = pd.merge(
    flights_df,
    weather_data.to_hourly_dataframe(),
    left_on=['queried_airport_iata', 'date'],
    right_on=['queried_airport_code', 'date'],
    how='left'
)
merged_df

,direction,date,queried_airport_iata,snowfall,rain,precipitation,wind_speed_10m,wind_gusts_10m,cloud_cover_low,cloud_cover,temperature_2m,apparent_temperature,surface_pressure,relative_humidity_2m,pressure_msl,queried_airport_code
0,Arrival,2025-10-25 19:00:00-04:00,ATL,0.0,0.0,0.0,12.781267,29.160000,0.0,100.0,17.25,13.933897,987.797729,40.934200,1023.400024,ATL
1,Arrival,2025-10-26 00:00:00-04:00,ATL,0.0,0.0,0.0,15.121070,30.599998,0.0,100.0,13.95,10.177357,988.940796,48.473717,1025.000000,ATL
2,Arrival,2025-10-25 23:00:00-04:00,ATL,0.0,0.0,0.0,14.830076,32.039997,0.0,100.0,14.65,10.938719,988.930481,46.656029,1024.900024,ATL
3,Arrival,2025-10-26 00:00:00-04:00,ATL,0.0,0.0,0.0,15.121070,30.599998,0.0,100.0,13.95,10.177357,988.940796,48.473717,1025.000000,ATL
4,Arrival,2025-10-26 00:00:00-04:00,ATL,0.0,0.0,0.0,15.121070,30.599998,0.0,100.0,13.95,10.177357,988.940796,48.473717,1025.000000,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL
764,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL
765,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL
766,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL


Now we have a merged dataset that combines both flight and weather data. This dataset can be used for further analysis, such as predicting flight delays based on weather conditions.

We need the counts of departure and arrival flights for a given date and airport, so we can use the following methods:

In [83]:
flights.get_count_airport_flights()

768

We can also get the hourly counts for each hour of the day using the following methods:

In [84]:
flights.get_hourly_airport_flights_count()

,date,flight_count
0,2025-10-25 19:00:00-04:00,1
1,2025-10-25 21:00:00-04:00,1
2,2025-10-25 22:00:00-04:00,3
3,2025-10-25 23:00:00-04:00,3
4,2025-10-26 00:00:00-04:00,6
5,2025-10-26 01:00:00-04:00,1
6,2025-10-26 02:00:00-04:00,4
7,2025-10-26 03:00:00-04:00,3
8,2025-10-26 05:00:00-04:00,21
9,2025-10-26 06:00:00-04:00,39


In [86]:
mergeed_df2 = pd.merge(
    merged_df,
    flights.get_hourly_airport_flights_count(),
    left_on=['date'],
    right_on=['date'],
    how='left'
)
mergeed_df2

,direction,date,queried_airport_iata,snowfall,rain,precipitation,wind_speed_10m,wind_gusts_10m,cloud_cover_low,cloud_cover,temperature_2m,apparent_temperature,surface_pressure,relative_humidity_2m,pressure_msl,queried_airport_code,flight_count
0,Arrival,2025-10-25 19:00:00-04:00,ATL,0.0,0.0,0.0,12.781267,29.160000,0.0,100.0,17.25,13.933897,987.797729,40.934200,1023.400024,ATL,1
1,Arrival,2025-10-26 00:00:00-04:00,ATL,0.0,0.0,0.0,15.121070,30.599998,0.0,100.0,13.95,10.177357,988.940796,48.473717,1025.000000,ATL,6
2,Arrival,2025-10-25 23:00:00-04:00,ATL,0.0,0.0,0.0,14.830076,32.039997,0.0,100.0,14.65,10.938719,988.930481,46.656029,1024.900024,ATL,3
3,Arrival,2025-10-26 00:00:00-04:00,ATL,0.0,0.0,0.0,15.121070,30.599998,0.0,100.0,13.95,10.177357,988.940796,48.473717,1025.000000,ATL,6
4,Arrival,2025-10-26 00:00:00-04:00,ATL,0.0,0.0,0.0,15.121070,30.599998,0.0,100.0,13.95,10.177357,988.940796,48.473717,1025.000000,ATL,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL,102
764,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL,102
765,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL,102
766,Departure,2025-10-26 11:00:00-04:00,ATL,0.0,0.0,0.0,15.568700,36.000000,100.0,100.0,13.85,10.278877,989.121582,53.641556,1025.199951,ATL,102


This concludes the data retrieval process using OpenMeteo and AeroDataBox APIs. The retrieved weather and flight data can now be used for further analysis and modeling to predict flight delays based on various factors.